In [169]:
from os import urandom
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from hashlib import sha256
import json
from secrets import randbits

flag = "this is a random flag yeah"

A = [randbits(64) for _ in range(48)]
a = randbits(48)

b = 0
for i in range(48):
    if (a>>i)%2 == 1:
        b ^= A[i]

hsh = sha256(str(a).encode())
key = hsh.digest()
iv = urandom(16)
cipher = AES.new(key, AES.MODE_CBC, IV=iv)
out = {'A':A, 'b':b, 'c':cipher.encrypt(pad(flag.encode(),16)).hex()+';'+iv.hex()}
print(json.dumps(out))
B = 0
for wild in A:
    B ^= wild


{"A": [10267453770467260924, 13071561876500477983, 3210365979343089945, 5145029468585073443, 15814324890382167010, 5456091749275627318, 9368509311968460437, 2331052922399088586, 15621141568741049013, 16659139774675541472, 1135404811578433259, 16417707277713845125, 13443979217491709268, 3540176713291939097, 15141098798953498092, 2827999829117199841, 15138114957387384118, 18294573322253832905, 4856852260539080126, 6513353756275495803, 17410688749956725058, 13485612021583528606, 15779480869917977489, 15092482891174352047, 9761845908731602688, 3613364561303402004, 12670180446659253442, 3481237650310167004, 4024016248652051683, 9560971345180055780, 8649504439949283860, 10091458660881754039, 17260269980967850266, 8046553069223982597, 12513079730068617917, 4356335129354069703, 7101747299031504502, 16528924146394592553, 16992065017205184362, 15465731798529354377, 11404954312286339310, 2941422602733095987, 9198125715907598069, 17458665110675970676, 12512282563595635157, 12593826019589313025, 31

Known:

- A
- b 
- c = ENC() + IV

In [9]:
a = int("10000001",2)
list(i for i in range(48) if (a>>i)%2 == 1)

[0, 7]

Known: $A = [rndint(64)]*48$

$a =  rndint(48)$

$key = SHA(a)$

Known: $b = XOR^{48}_{i=0} A[i] * (\text{a i'th bit is set})$

```
b = 0
for i in range(48):
    if (a>>i)%2 == 1:
        b ^= A[i]
```

Known $B$

```
B = 0
for wild in A:
    B ^= wild
```

In [319]:
example = {"A": [16020352969324620574, 5251799175822576953, 12296626828365797874, 16263694180239238901, 5385636492353288931, 13816899847512435959, 7742743182745056615, 14957880546606728616, 16238960719716473046, 9511420477454565411, 9512475460211815823, 1512387297968834613, 11025347025977204713, 3442476855913255254, 6772463291312668819, 16338156216606160884, 2236098207358598727, 376086842397216431, 15268871214872582923, 16136676900066230671, 18041385868476951874, 11442980370112042200, 7802644279583543299, 8878651250728940760, 14568762204255907208, 17962654719168969135, 8052556424490470345, 14768970076962522414, 4368685111649028024, 15796814169905729571, 5276492083840585869, 17263640889969997431, 14934405837334166129, 10022892872091966820, 16699543543288764170, 14622209536502362273, 7554713645004227559, 8135446121655354220, 3350332388256634476, 17429636428790492991, 6595155937027409855, 7961841859889295885, 12122064162111098587, 18378838582646317899, 2028853019434628259, 92499015470528509, 10815510368964364000, 13037305624228890100], "b": 15573269927557336467, "c": "daff70e654460d3038c0336ff21a75ea9b5517d86a4ebde7c8ab605fc2edb75a;5c1ff11ac0a1763fb36f0f33724d1982"}

# Idea: solve it via z3

Takes to long tho ./

In [ ]:
from z3 import *
set_param("parallel.enable", True)
A = [BitVecVal(ai, 64) for ai in example["A"]]
a = BitVec('a', 48)

b = BitVecVal(0, 64)
for i in range(48):
    bit = LShR(a, i) & 1
    b = If(bit == 1, b ^ A[i], b)

s = Solver()
s.add(b == BitVecVal(example["b"], 64))

# Solve
if s.check() == sat:
    model = s.model()
    print(f"Solution found: a = {model[a].as_long()}")
else:
    print("No solution found.")

# Solve it via gaussian elimination

This roughyl corresponds to 

$A * X = B$, we know $A$ and $B$ and want to know $X$

In terms of dimensions, $A \in \mathbb{F}_2^{48\cdot 64}$, $B \in \mathbb{F}_2^{64}$, we want to find $X \in \mathbb{F}_2^{48}$

The rule is $A(m*n) * X(n*p) = B(m*p) \implies A(64\cdot 48) * X(48\cdot 1) = B(64\cdot 1)$

In [320]:
example = {"A": [11546540537528617495, 3953513797278791755, 16516202401637121521, 6480091461691686298, 5415621636411405885, 10628343542452056221, 16804857524268894137, 11070777742689911487, 10881535484251511012, 17966046375983488023, 92351007324335235, 2665221146797899433, 18308056028252502743, 3553276075825537305, 13384922605600306491, 3048888282492555516, 1908801876994191179, 9852353140807672675, 8625960673999586837, 4409787929871116825, 17514601379465241187, 9904168025566413937, 2728277318533081862, 6010859725966145270, 5622096531523508386, 16747223852604765651, 15555407015417990821, 11163629809962300268, 11578463030468617300, 2358948103157706282, 4029525561681691014, 15730716774305376486, 15963277504925092441, 923580729402267087, 16579142232180486013, 18396212619159151767, 11370524598900747, 5385202585588200830, 884635894057167401, 5117844456277346894, 16100002806120224902, 102048255797513210, 256501806069851261, 12870170846296721366, 15503421377502358280, 17086639866599811250, 11574268021855107273, 11491254951168319333], "b": 2722595893016982156, "c": "a3e0613e7eb30532f502ba00fffdc06fe9567ff252d85bbdb7344abebca419b1;4185ec07c6c2422da4c626bf7ea438a8"}

In [ ]:
from os import urandom
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from hashlib import sha256
from secrets import randbits

flag = "this is a random flag yeah"

A = [randbits(64) for _ in range(48)]
a = randbits(48)

test = []
b = 0
for i in range(48):
    if (a>>i)%2 == 1:
        test.append(1)
        b ^= A[i]
    else:
        test.append(0)

hsh = sha256(str(a).encode())
key = hsh.digest()
iv = urandom(16)
cipher = AES.new(key, AES.MODE_CBC, IV=iv)
example = {'A':A, 'b':b, 'c':cipher.encrypt(pad(flag.encode(),16)).hex()+';'+iv.hex()}


# result vector is reversed here ...
a,bin(a)[2:],"".join(str(x) for x in test)[::-1]

(98984007035714,
 '10110100000011010000010100100111110101101000010',
 '010110100000011010000010100100111110101101000010')

In [321]:
from sage.all import * 

In [322]:
gf2 = GF(2)
m = 48
n = 64
p = 1

In [323]:
def toVec(input,size):
    return list(int(x) for x in bin(input)[2:].rjust(size,"0"))

In [324]:
mat = Matrix(gf2,m,n)
for i,ai in enumerate(example["A"]):
    mat[i] = vector(gf2, toVec(ai,64))
mat = mat.transpose()
mat

64 x 48 dense matrix over Finite Field of size 2

In [326]:
B = vector(gf2, toVec(example["b"],64))
B.length()

64

In [327]:
res = mat.solve_right(B)
res = int("".join([str(bit) for bit in res[::-1]]),2)

assert res.bit_length() <= 48

In [328]:
res,bin(res)

(196149404513851, '0b101100100110010110010111100101000011011000111011')

In [329]:
hsh = sha256(str(res).encode())
key = hsh.digest()
ciphertext,iv = example["c"].split(";",maxsplit=1)
cipher = AES.new(key, AES.MODE_CBC, IV=bytes.fromhex(iv))
cipher.decrypt(bytes.fromhex(ciphertext))

b'ictf{XOR_m337_in_th3_m1ddl3}\x04\x04\x04\x04'